### 머신러닝 두번째 - 전처리 데이터 이용 머신러닝

오라클 접속 및 필터링된 데이터 불러오기

In [25]:
import oracledb
import os
import pandas as pd
import numpy as np

oracledb.init_oracle_client(lib_dir=r"C:\Users\user\Desktop\KSM\Tools\instantclient-basic-windows.x64-19.25.0.0.0dbru\instantclient_19_25")

# 1. Oracle 연결 (Thin 모드 사용 예시)
conn = oracledb.connect(
    user="libra",
    password="ksm0923",
    dsn="localhost:1521/XE",  # 예: "localhost:1521/XEPDB1"
    mode=oracledb.DEFAULT_AUTH
)

# 2. FILTERED 테이블 불러오기
query = "SELECT * FROM LIBRA.FILTERED"
df = pd.read_sql(query, con=conn)

# 3. 숫자형 컬럼만 추출
df_numeric = df.select_dtypes(include=["int", "float", "object"])

# 4. NumPy 배열로 변환
data_array = df_numeric.to_numpy()

# 5. 숫자처럼 처리할 컬럼 추출
df_checked = df.copy()
text_cols = ["SNM", "STYP", "FND", "RGN", "USC"]
target_cols = [col for col in df.columns if col not in text_cols]

for col in target_cols:
    try:
        # 쉼표, 공백 제거 등 전처리 추가 가능
        cleaned = df[col].astype(str).str.replace(",", "", regex=False).str.strip()

        # 숫자 변환 시도 (실패 시 예외 발생)
        converted = pd.to_numeric(cleaned, errors="raise")

        # 성공 시 반영
        df_checked[col] = converted

    except Exception as e:
        print(f"❌ [ERROR] 컬럼 '{col}' 변환 실패 → {e}")
        raise SystemExit(f"🚨 데이터 변환 중단: '{col}' 열에 숫자가 아닌 값이 존재합니다.")



C:\Users\user\AppData\Local\Temp\ipykernel_16240\1727712138.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=conn)


문자형 데이터 중 숫자로 이루어진 데이터 추출 및 숫자형으로 변환작업 확인

In [26]:
# 확인용 컬럼별 데이터 타입 출력
print("📌 컬럼별 데이터 타입:")
for col in df_checked.columns:
    print(f"{col:20} ➤ {df_checked[col].dtype}")

# 숫자형 / 문자형 컬럼 분리 확인
numeric_cols = df_checked.select_dtypes(include=["number"]).columns.tolist()
object_cols = df_checked.select_dtypes(include=["object"]).columns.tolist()

print("\n🔢 숫자형 컬럼:", numeric_cols)
print(f"숫자형 컬럼 갯수: {len(numeric_cols)}개")

print("🔤 문자형 컬럼:", object_cols)
print(f"문자형 컬럼 갯수: {len(object_cols)}개")


📌 컬럼별 데이터 타입:
YR                   ➤ int64
RK                   ➤ float64
SNM                  ➤ object
SCR                  ➤ float64
ID                   ➤ float64
STYP                 ➤ object
FND                  ➤ object
RGN                  ➤ object
USC                  ➤ object
APS_APS              ➤ float64
BGT_MCT              ➤ int64
BGT_UBGT             ➤ int64
BPS_BPS              ➤ float64
BR_BCNT_SUM          ➤ int64
CPSS_CPS             ➤ float64
EHS_EHS              ➤ float64
FACLT_EQP_TPC        ➤ int64
FACLT_LAS            ➤ float64
FACLT_RS_TRS         ➤ int64
LBRT_USTL            ➤ int64
LPK_LPK              ➤ float64
LPS_LPS              ➤ float64
LS_LB_SUM            ➤ int64
LS_LU_SUM            ➤ int64
SPK_SPK              ➤ float64
STL_MCT              ➤ int64
STL_USTL             ➤ int64
VPS_VPS              ➤ float64
VUC_UC_LUC           ➤ int64

🔢 숫자형 컬럼: ['YR', 'RK', 'SCR', 'ID', 'APS_APS', 'BGT_MCT', 'BGT_UBGT', 'BPS_BPS', 'BR_BCNT_SUM', 'CPSS_CPS', 'EHS_EH

In [29]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np


# 1. X, y 분리 (예: df_checked 사용)
X = df_checked[['APS_APS', 'BGT_MCT', 'BGT_UBGT', 'BPS_BPS', 'BR_BCNT_SUM', 'CPSS_CPS', 'EHS_EHS', 'FACLT_EQP_TPC', 'FACLT_LAS', 'FACLT_RS_TRS', 'LBRT_USTL', 'LPK_LPK', 'LPS_LPS', 'LS_LB_SUM', 'LS_LU_SUM', 'SPK_SPK', 'STL_MCT', 'STL_USTL', 'VPS_VPS', 'VUC_UC_LUC']]
y = df_checked["SCR"]

# 2. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 모델 훈련
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# 4. 평가
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("✅ RMSE:", rmse)

✅ RMSE: 7.871503365536022


In [65]:
input_columns = [
    "APS_APS", "BGT_MCT", "BGT_UBGT", "BPS_BPS", "BR_BCNT_SUM",
    "CPSS_CPS", "EHS_EHS", "FACLT_EQP_TPC", "FACLT_LAS", "FACLT_RS_TRS",
    "LBRT_USTL", "LPK_LPK", "LPS_LPS", "LS_LB_SUM", "LS_LU_SUM",
    "SPK_SPK", "STL_MCT", "STL_USTL", "VPS_VPS", "VUC_UC_LUC"
]

def predict_scr_by_school_name(school_name):
    query = f"SELECT * FROM LIBRA.NUM06_2014 WHERE TO_CHAR(SNM) = '{school_name}'"
    df_new = pd.read_sql(query, con=conn)

    if df_new.empty:
        raise ValueError(f"❗ 대학 '{school_name}'에 해당하는 데이터가 존재하지 않습니다.")

    
    # 전처리 (쉼표 제거, strip 등)
    for col in input_columns:
        df_new[col] = (
            df_new[col]
            .astype(str)
            .str.replace(",", "", regex=False)
            .str.strip()
        )
        df_new[col] = pd.to_numeric(df_new[col], errors="raise")

    missing_cols = [col for col in input_columns if col not in df_new.columns]
    if missing_cols:
        raise ValueError(f"❗ 다음 컬럼이 누락되어 예측이 불가능합니다: {missing_cols}")


    # 입력 피처 추출
    X_new = df_new[input_columns]

    # 예측
    predicted_score = model.predict(X_new)[0]
    print(f"🎯 예측된 SCR 점수 ({school_name}): {predicted_score:.2f}")
    return X_new

In [69]:
X_new = predict_scr_by_school_name("서울대학교")
print(X_new)
print("🧪 실행 쿼리:", query)

print(X_new[input_columns])

🎯 예측된 SCR 점수 (서울대학교): 63.77
   APS_APS      BGT_MCT      BGT_UBGT     BPS_BPS  BR_BCNT_SUM  CPSS_CPS  \
0      1.7  10008392000  742116268000  167.899994      4699851  333642.1   

     EHS_EHS  FACLT_EQP_TPC  FACLT_LAS  FACLT_RS_TRS     LBRT_USTL  LPK_LPK  \
0  53.299999            339      47828          6557  712500000000      3.8   

   LPS_LPS  LS_LB_SUM  LS_LU_SUM  SPK_SPK     STL_MCT      STL_USTL  \
0     16.1     548783      31877        4  9337309020  712500000000   

      VPS_VPS  VUC_UC_LUC  
0  158.300003     4431167  
🧪 실행 쿼리: SELECT * FROM LIBRA.FILTERED
   APS_APS      BGT_MCT      BGT_UBGT     BPS_BPS  BR_BCNT_SUM  CPSS_CPS  \
0      1.7  10008392000  742116268000  167.899994      4699851  333642.1   

     EHS_EHS  FACLT_EQP_TPC  FACLT_LAS  FACLT_RS_TRS     LBRT_USTL  LPK_LPK  \
0  53.299999            339      47828          6557  712500000000      3.8   

   LPS_LPS  LS_LB_SUM  LS_LU_SUM  SPK_SPK     STL_MCT      STL_USTL  \
0     16.1     548783      31877        

C:\Users\user\AppData\Local\Temp\ipykernel_16240\3482769199.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_new = pd.read_sql(query, con=conn)


In [39]:
print(y.describe())

count    533.000000
mean      64.596398
std       12.146388
min       42.520000
25%       49.840000
50%       69.000000
75%       72.600000
max       86.700000
Name: SCR, dtype: float64


In [40]:
print(X_train.head())
print(X_train.std())  # 표준편차 0이면 문제!

     APS_APS     BGT_MCT      BGT_UBGT     BPS_BPS  BR_BCNT_SUM  CPSS_CPS  \
298      1.5  4816440000  349899040000  122.199997      3404347  177966.0   
403      0.0   250000000   25541708197   86.699997       111142  202476.0   
447      0.7  2342492720  197592086000   72.599998      1212065  137578.9   
311      1.4  1565804000  156588915000   99.900002      1387026  112645.9   
140      1.3  2740000000  296215957000   78.599998      1619607  148213.3   

       EHS_EHS  FACLT_EQP_TPC     FACLT_LAS  FACLT_RS_TRS     LBRT_USTL  \
298  36.299999            190  41434.398438          5478  357671453238   
403   0.000000              0      0.000000             0   23776984973   
447  50.599998             20  11575.000000          2502  228040965881   
311  92.500000            192  19671.199219          2838  162244713106   
140  49.400002            392  26533.000000          3930  267846212000   

     LPK_LPK  LPS_LPS  LS_LB_SUM  LS_LU_SUM  SPK_SPK     STL_MCT  \
298      1.7      

NameError: name 'X_new' is not defined